In [3]:
import numpy as np
from pyamg.gallery import diffusion_stencil_2d, stencil_grid
from pyamg.classical.split import RS, PMIS, MIS
from scipy.io import mmwrite
from scipy.sparse import csr_matrix
import PetscBinaryIO

In [4]:
def PetscPrint(filename, Mat):
    PrintMat = csr_matrix(Mat)
    PetscBinaryIO.PetscBinaryIO().writeMatSciPy(open(filename,'w'), PrintMat)

In [4]:
def ClassicStrength(A, theta = 0.0):
    A = A.tocsr()
    
    Sp = np.empty_like(A.indptr)
    Sj = np.empty_like(A.indices)
    Sx = np.empty_like(A.data)
    
    nnz = 0
    n = A.shape[0]
    
    Sp[0] = 0
    for i in range(0, n):
        diag = 0
        for j in range(A.indptr[i], A.indptr[i+1]):
            if (A.indices[j] == i):
                diag = A.data[j]
                break
        
        if diag < 0:
            row_scale = -np.inf
            for j in range(A.indptr[i], A.indptr[i+1]):
                val = A.data[j]
                if (val > row_scale):
                    row_scale = val
        else:
            row_scale = np.inf
            for j in range(A.indptr[i], A.indptr[i+1]):
                val = A.data[j]
                if (val < row_scale):
                    row_scale = val
        
        threshold = row_scale * theta
        
        Sj[nnz] = i
        Sx[nnz] = diag
        nnz += 1
        
        if diag < 0:
            for j in range(A.indptr[i], A.indptr[i+1]):
                val = A.data[j]
                if val > threshold:
                    col = A.indices[j]
                    Sj[nnz] = col
                    Sx[nnz] = val
                    nnz += 1
        else:
            for j in range(A.indptr[i], A.indptr[i+1]):
                val = A.data[j]
                if val < threshold:
                    col = A.indices[j]
                    Sj[nnz] = col
                    Sx[nnz] = val
                    nnz += 1
                    
        Sp[i + 1] = nnz

    return csr_matrix((Sx, Sj, Sp), shape=A.shape)

In [5]:
def SymmetricStrength(A, theta = 0.0):
    A = A.tocsr()
    
    Sp = np.empty_like(A.indptr)
    Sj = np.empty_like(A.indices)
    Sx = np.empty_like(A.data)
    
    n = A.shape[0]
    
    diags = np.zeros(n)
    row_thresholds = np.zeros(n)

    nnz = 0
    Sp[0] = 0
    for i in range(0, n):
        diag = 0
        for j in range(A.indptr[i], A.indptr[i+1]):
            if (A.indices[j] == i):
                diags[i] = A.data[j]
                break
        
        if diag < 0:
            row_scale = -np.inf
            for j in range(A.indptr[i], A.indptr[i+1]):
                val = A.data[j]
                if (val > row_scale):
                    row_scale = val
        else:
            row_scale = np.inf
            for j in range(A.indptr[i], A.indptr[i+1]):
                val = A.data[j]
                if (val < row_scale):
                    row_scale = val
        
        row_thresholds[i] = row_scale * theta
    
    for i in range(0, n):
        Sj[nnz] = i
        Sx[nnz] = diags[i]
        nnz += 1
        
        if diag < 0:
            for j in range(A.indptr[i], A.indptr[i+1]):
                val = A.data[j]
                col = A.indices[j]
                if val > row_thresholds[i] or val > row_thresholds[col]:
                    Sj[nnz] = col
                    Sx[nnz] = val
                    nnz += 1
        else:
            for j in range(A.indptr[i], A.indptr[i+1]):
                val = A.data[j]
                col = A.indices[j]
                if val < row_thresholds[i] or val < row_thresholds[col]:
                    Sj[nnz] = col
                    Sx[nnz] = val
                    nnz += 1
                    
        Sp[i + 1] = nnz

    return csr_matrix((Sx, Sj, Sp), shape=A.shape)

In [6]:
def ModClassicalInterp(A, S, splitting):
    A = A.tocsr()
    S = S.copy()
    S = S.tocsr()

    Pp = np.empty_like(A.indptr)
    
    col_to_new = np.empty(A.shape[0])
    col_to_new.fill(-1)

    nnz = 0
    Pp[0] = nnz
    for i in range(A.shape[0]):
        if splitting[i] == 1:
            nnz += 1
        else:
            for j in range(S.indptr[i], S.indptr[i+1]):
                col = S.indices[j]
                if splitting[col] == 1:
                    nnz += 1
        Pp[i+1] = nnz
                
    Pj = np.empty(nnz, dtype=Pp.dtype)
    Px = np.empty(nnz, dtype=A.dtype)
    
    nnz = 0
    ctr = 0
    
    A.sort_indices()
    S.sort_indices()
    
    row_strong = np.zeros(A.shape[0], dtype='float')
    pos = np.empty(A.shape[0], dtype='int')
    pos.fill(-1)
            
    n_cols = 0
    for i in range(A.shape[0]):
        if splitting[i] == 1:
            Pj[nnz] = i
            Px[nnz] = 1.0
            col_to_new[i] = n_cols
            n_cols += 1
            nnz += 1
        else:            
            # Add coarse points in row
            ctr = S.indptr[i]
            endS = S.indptr[i+1]
            sign = 1
            weak_sum = 0
            for j in range(A.indptr[i], A.indptr[i+1]):
                col = A.indices[j]
                val = A.data[j]
                if ctr < endS and S.indices[ctr] == col:
                    if splitting[col] == 1:
                        pos[col] = nnz
                        Pj[nnz] = col
                        Px[nnz] = val
                        nnz += 1
                    elif col == i:
                        weak_sum += val
                        if val < 0:
                            sign = -1
                    else:
                        row_strong[col] = val
                    ctr += 1
                else:
                    weak_sum += val
    
            # Add distance two to coarse sum 
            for j in range(S.indptr[i], S.indptr[i+1]):
                col = S.indices[j]
                if col == i or splitting[col] == 1:
                    continue
                coarse_sum = 0
                for k in range(A.indptr[col], A.indptr[col+1]):
                    colk = A.indices[k]
                    if pos[colk] >= 0:
                        val = A.data[k]
                        if val * sign < 0:
                            coarse_sum += val
                if coarse_sum == 0:
                    weak_sum += S.data[j]
                    row_strong[col] = 0
                else:
                    row_strong[col] /= coarse_sum
            
            for j in range(S.indptr[i], S.indptr[i+1]):
                col = S.indices[j]
                row_val = row_strong[col]
                if col == i or splitting[col] == 1:
                    continue
                
                if row_val != 0:
                    for k in range(A.indptr[col], A.indptr[col+1]):
                        colk = A.indices[k]
                        idx = pos[colk]
                        if idx >= 0:
                            val = A.data[k]
                            if val * sign < 0:
                                Px[idx] += row_val * val 
                                
            for j in range(S.indptr[i], S.indptr[i+1]):
                col = S.indices[j]
                idx = pos[col]
                if idx >= 0:
                    Px[idx] /= -weak_sum
                    pos[col] = -1
                else:
                    row_strong[col] = 0
                    
    for i in range(nnz):
        Pj[i] = col_to_new[Pj[i]]
                    
    return csr_matrix((Px, Pj, Pp), shape = (A.shape[0], n_cols))

In [20]:
def ExtendInterpolation(A, S, splitting):
    A = A.tocsr()
    S = S.copy()
    S = S.tocsr()
    A.sort_indices()
    S.sort_indices()
    
    Pp = np.empty_like(A.indptr)
    pos = np.empty(A.shape[0], dtype='int')
    pos.fill(-1)
    
    nnz = 0
    Pp[0] = nnz
    for i in range(A.shape[0]):
        if splitting[i] == 1:
            nnz += 1
        else:
            row_pos = list()
            for j in range(S.indptr[i], S.indptr[i+1]):
                col = S.indices[j]
                if col == i: 
                    continue
                
                # Add distance one and two connections
                if splitting[col] == 1:
                    if pos[col] == -1:
                        pos[col] = 1
                        row_pos.append(col)
                        nnz += 1
                else:
                    for k in range(S.indptr[col], S.indptr[col+1]):
                        colk = S.indices[k]
                        if splitting[colk] == 1 and pos[colk] == -1:
                            pos[colk] = 1
                            row_pos.append(colk)
                            nnz += 1
            for col in row_pos:
                pos[col] = -1
        Pp[i+1] = nnz
                
    Pj = np.empty(nnz, dtype=Pp.dtype)
    Px = np.empty(nnz, dtype=A.dtype)
    
    nnz = 0
    ctr = 0
    
    row_strong = np.zeros(A.shape[0], dtype='float')
            
    for i in range(A.shape[0]):
        if splitting[i] == 1:
            Pj[nnz] = i
            Px[nnz] = 1.0
            nnz += 1
        else:            
            # Add coarse points in row
            weak_sum = 0
            
            for j in range(S.indptr[i], S.indptr[i+1]):
                col = S.indices[j]
                val = S.data[j]   
                
                if col == i:
                    continue
                    
                # Add Distance-One Connections
                if splitting[col] == 1: 
                    if pos[col] == -1:
                        pos[col] = nnz
                        Pj[nnz] = col
                        Px[nnz] = val
                        nnz += 1
                    else:
                        Px[pos[col]] = val    
                else:
                    row_strong[col] = val
                        
                    # Add distance two connections
                    for k in range(S.indptr[col], S.indptr[col+1]):
                        colk = S.indices[k]
                        if splitting[colk] == 1 and pos[colk] == -1:
                            pos[colk] = nnz
                            Pj[nnz] = colk
                            Px[nnz] = 0
                            nnz += 1
            
            # Add weak connections not in C^_{i} to weak_sum
            weak_sum = 0
            ctr = S.indptr[i]
            endS = S.indptr[i+1]
            for j in range(A.indptr[i], A.indptr[i+1]):
                col = A.indices[j]
                val = A.data[j]  
                if ctr < endS and S.indices[ctr] == col:
                    ctr += 1
                    if col == i:
                        weak_sum += val
                else:
                    # Weak sum
                    if pos[col] == -1:
                        weak_sum += val
            
            # Add distance two to coarse sum 
            for j in range(S.indptr[i], S.indptr[i+1]):
                col = S.indices[j]
                
                # Find diag val
                for k in range(S.indptr[col], S.indptr[col+1]):
                    if S.indices[k] == col:
                        val = S.data[k]
                        break
                sign = 1
                if val < 0:
                    sign = -1
                    
                if col == i or splitting[col] == 1:
                    continue
                coarse_sum = 0
                for k in range(A.indptr[col], A.indptr[col+1]):
                    colk = A.indices[k]
                    if pos[colk] >= 0 or colk == i:
                        val = A.data[k]
                        if val * sign < 0:
                            coarse_sum += val
                if coarse_sum == 0:
                    weak_sum += S.data[j]
                else:
                    coarse_sum = row_strong[col] / coarse_sum
                    for k in range(A.indptr[col], A.indptr[col+1]):
                        colk = A.indices[k]
                        idx = pos[colk]
                        if idx >= 0:
                            val = A.data[k]
                            if val * sign < 0:
                                Px[idx] += coarse_sum * val 
                        if colk == i:
                            weak_sum += coarse_sum * val
            
            for j in range(S.indptr[i], S.indptr[i+1]):
                col = S.indices[j]
                row_strong[col] = 0
                                
            for j in range(Pp[i], Pp[i+1]):
                Px[j] /= -weak_sum
                col = Pj[j]
                pos[col] = -1

                            
    cols = np.zeros(nnz, dtype = 'int')
    for i in range(nnz):
        cols[i] = Pj[i]
    cols.sort()
    
    cols_to_new = np.empty(A.shape[0], dtype='int')
    cols_to_new.fill(-1)
    
    n_cols = 0
    col = cols[0]
    cols_to_new[col] = n_cols
    n_cols += 1
    for i in range(1, nnz):
        col = cols[i]
        if col != cols[n_cols-1]:
            cols_to_new[col] = n_cols
            cols[n_cols] = col
            n_cols += 1
            
    for i in range(nnz):
        Pj[i] = cols_to_new[Pj[i]]
                    
    return csr_matrix((Px, Pj, Pp), shape = (A.shape[0], n_cols))

In [21]:
# MIS-2 Algorithm
def mis2(A):
    n = A.shape[0]
    
    # 1
    np.random.seed(100) # For comparisons
    r = np.random.rand(n)
    
    #2
    A_dir = A.copy().tocsr()
    for i in range(n):
        for j in range(A_dir.indptr[i], A_dir.indptr[i+1]):
            col = A_dir.indices[j]
            if (r[i] <= r[col]):
                A_dir.data[j] = 0.0
    A_dir.eliminate_zeros()

    # 3
    S = []
    
    # 4
    V = np.arange(n)
    states = np.full((n,1), -1)
    remaining = n
        
    iterate = 0
    
    while remaining > 0:
        # 6
        print("Iteration", iterate, "# active nodes", len(V), "/", n, ", MIS2 set size is", len(S))
        iterate+=1
        L = []
        for v in V:
            found = False
            for jj in range(A_dir.indptr[v], A_dir.indptr[v+1]):
                w = A_dir.indices[jj]
                if states[w] < 0:
                    found = True
                    break
            if not found:
                L.append(v)
                states[v] = -2
                        
        # 8
        for v in L:
            found = False
            for jj in range(A.indptr[v], A.indptr[v+1]):
                w = A.indices[jj]
                for kk in range(A.indptr[w], A.indptr[w+1]):
                    u = A.indices[kk]
                    if states[u] < -1 and r[u] > r[v]:
                        found = True
                        break
            if not found:
                S.append(v)
                states[v] = -3                    
        
        # 11
        for v in V:
            if states[v] == -3:
                continue
            found = False
            for jj in range(A.indptr[v], A.indptr[v+1]):
                u = A.indices[jj]
                if states[u] == -3:
                    found = True
                    break
            if not found:
                for jj in range(A.indptr[v], A.indptr[v+1]):
                    w = A.indices[jj]
                    for kk in range(A.indptr[w], A.indptr[w+1]):
                        u = A.indices[kk]
                        if states[u] == -3:
                            found = True
                            break
                    if found:
                        break
            if found:
                states[v] = -4
            
        ctr = 0
        for i in range(remaining):
            v = V[i]
            if (states[v] == -4):
                states[v] = 0
            elif (states[v] == -3):
                states[v] = 1
            else:
            #if (states[V[i]] < 0):
                V[ctr] = V[i]
                ctr += 1
        V.resize(ctr)
        remaining = ctr
        
    print("Final MIS2 set size is", len(S))

    return S

In [22]:
################################
###    Gallery Tests
###############################
# Rotated Anisotropic Diffusion
n = 25
grid = (n,n)
sten = diffusion_stencil_2d(epsilon=0.001, theta=np.pi/8)
Aniso = stencil_grid(sten, grid, dtype='float').tocsr()
PetscPrint("aniso.pm", Aniso)

# Laplacian 27pt
n = 10
grid = (n,n,n)
sten = [[[-1,-1,-1],[-1,-1,-1],[-1,-1,-1]],
       [[-1,-1,-1],[-1,26,-1],[-1,-1,-1]],
       [[-1,-1,-1],[-1,-1,-1],[-1,-1,-1]]]
Laplacian = stencil_grid(sten, grid, dtype='float').tocsr()
PetscPrint("laplacian.pm", Laplacian)

In [23]:
################################
###    Core Tests
###############################
# Transposes
AT = csr_matrix(Aniso)
AT.T
PetscPrint("aniso_T.pm", AT)

AT = csr_matrix(Laplacian)
AT.T
PetscPrint("laplacian_T.pm", AT)

In [24]:
################################
###    Strength Tests
###############################
Aniso_S = ClassicStrength(Aniso, 0.25)
PetscPrint("aniso_S.pm", Aniso_S)

Laplacian_S = ClassicStrength(Laplacian, 0.25)
PetscPrint("laplacian_S.pm", Laplacian_S)

Aniso_SS = SymmetricStrength(Aniso, 0.25)
PetscPrint("aniso_SS.pm", Aniso_SS)

Laplacian_SS = SymmetricStrength(Laplacian, 0.25)
PetscPrint("laplacian_SS.pm", Laplacian_SS)

In [25]:
################################
###    Ruge Stuben AMG Tests
###############################  
# RS Splitting
AnisoSplit = RS(Aniso_S)
np.savetxt("aniso_split.txt", AnisoSplit, fmt="%d")
LaplacianSplit = RS(Laplacian_S)
np.savetxt("laplacian_split.txt", LaplacianSplit, fmt="%d")

# Direct Interpolation
from pyamg.classical.interpolate import direct_interpolation
Aniso_P = direct_interpolation(Aniso, Aniso_S, AnisoSplit)
PetscPrint("aniso_P_direct.pm", Aniso_P)
Laplacian_P = direct_interpolation(Laplacian, Laplacian_S, LaplacianSplit)
PetscPrint("laplacian_P_direct.pm", Laplacian_P)

# Modified Classical Interpolation
Aniso_P = ModClassicalInterp(Aniso, Aniso_S, AnisoSplit)
PetscPrint("aniso_P_mod_class.pm", Aniso_P)
Laplacian_P = ModClassicalInterp(Laplacian, Laplacian_S, LaplacianSplit)
PetscPrint("laplacian_P_mod_class.pm", Laplacian_P)

# Extended+i Interpolation
Aniso_P = ExtendInterpolation(Aniso, Aniso_S, AnisoSplit)
PetscPrint("aniso_P_extend.pm", Aniso_P)
Laplacian_P = ExtendInterpolation(Laplacian, Laplacian_S, LaplacianSplit)
PetscPrint("laplacian_P_extend.pm", Laplacian_P)
print(Laplacian_P.shape, Laplacian_P.nnz)

(1000, 125) 8792


In [26]:
################################
###    Smoothed Aggregation Tests
###############################        
# Test MIS
Aniso = Aniso.tocsr()
S = mis2(Aniso)
S = np.array(S)
split_Aniso = np.zeros(Aniso.shape[0])
for s in S:
    split_Aniso[s] = 1
np.savetxt("aniso_mis.txt", split_Aniso, fmt="%d")

Laplacian = Laplacian.tocsr()
S = mis2(Laplacian)
S = np.array(S)
split_Laplacian =  np.zeros(Laplacian.shape[0])
for s in S:
    split_Laplacian[s] = 1
np.savetxt("laplacian_mis.txt", split_Laplacian, fmt="%d")

Iteration 0 # active nodes 625 / 625 , MIS2 set size is 0
Iteration 1 # active nodes 152 / 625 , MIS2 set size is 31
Iteration 2 # active nodes 59 / 625 , MIS2 set size is 44
Iteration 3 # active nodes 17 / 625 , MIS2 set size is 50
Iteration 4 # active nodes 8 / 625 , MIS2 set size is 51
Iteration 5 # active nodes 2 / 625 , MIS2 set size is 52
Final MIS2 set size is 53
Iteration 0 # active nodes 1000 / 1000 , MIS2 set size is 0
Iteration 1 # active nodes 109 / 1000 , MIS2 set size is 19
Iteration 2 # active nodes 31 / 1000 , MIS2 set size is 27
Iteration 3 # active nodes 1 / 1000 , MIS2 set size is 33
Final MIS2 set size is 34


In [17]:
##############################
###  Random Matrix
##############################
from pyamg.gallery import sprand
R = sprand(100, 100, 0.04)
PetscPrint("random.pm", R)
x = np.ones((R.shape[0],))
b = R * x
np.savetxt("random_ones_b.txt", b, fmt="%f")
b = R.T * x
np.savetxt("random_ones_b_T.txt", b, fmt="%f")
x = np.arange(R.shape[0])
b = R * x
np.savetxt("random_inc_b.txt", b, fmt="%f")
b = R.T * x
np.savetxt("random_inc_b_T.txt", b, fmt="%f")